In [1]:
import _thread as thread

不建议使用 thread 模块。给出这个建议有很多原因，其中最明显的一个原因是在主线程退出之后，所有 其他线程都会在没有清理的情况下直接退出。而另一个模块 threading 会确保在所有“重要的” 子线程退出前，保持整个进程的存活（对于“重要的”这个含义的说明，请阅读下面的核心 提示：“避免使用 thread 模块”）。

Python 提供了多个模块来支持多线程编程，包括 thread、threading 和 Queue 模块等。程序是可以使用 thread 和 threading 模块来创建与管理线程。
thread 模块提供了基本的线程和锁 定支持；
而threading 模块提供了更高级别、功能更全面的线程管理。
使用 Queue 模块，用户 可以创建一个队列数据结构，用于在多线程之间进行共享

### thread 模块和锁对象 

In [ ]:
thread 模块的函数 
start_new_thread (function, args, kwargs=None) 派生一个新的线程，使用给定的 args 和可选的 kwargs 来执行 function 
allocate_lock() 分配 LockType 锁对象 
exit() 给线程退出指令 

LockType 锁对象的方法 
acquire (wait=None) 尝试获取锁对象 
locked () 如果获取了锁对象则返回 True，否则，返回 False 
release () 释放锁  

In [2]:
import _thread as thread
from time import sleep, ctime

loops = [4, 2]

def loop(nloop, nsec, lock):
    print ('start loop', nloop, 'at:', ctime())
    sleep(nsec)
    print ('loop', nloop, 'done at:', ctime())
    lock.release()

def main():
    print ('starting threads...')
    locks = []
    nloops = range(len(loops))

    for i in nloops:
        lock = thread.allocate_lock()
        lock.acquire()
        locks.append(lock)

    for i in nloops:
        thread.start_new_thread(loop, 
            (i, loops[i], locks[i]))

    for i in nloops:
        while locks[i].locked(): pass

    print ('all DONE at:', ctime())

if __name__ == '__main__':
    main()

starting threads...
start loop 0 at: Fri Jul  7 14:34:24 2017
start loop 1 at: Fri Jul  7 14:34:24 2017
loop 1 done at: Fri Jul  7 14:34:26 2017
loop 0 done at: Fri Jul  7 14:34:28 2017
all DONE at: Fri Jul  7 14:34:28 2017


在每个线程执行完成时，它会释放自己的锁对象。最后一个循环只是坐在那里等待（暂 停主线程），直到所有锁都被释放之后才会继续执行。因为我们按照顺序检查每个锁，所有可 能会被排在循环列表前面但是执行较慢的循环所拖累。这种情况下，大部分时间是在等待最 前面的循环。当这种线程的锁被释放时，剩下的锁可能早已被释放（也就是说，对应的线程 已经执行完毕）。结果就是主线程会飞快地、没有停顿地完成对剩下锁的检查。最后，你应该 知道只有当我们直接调用这个脚本时，最后几行语句才会执行 main()函数。 

### threading 模块 

#### threading 模块的对象 

In [ ]:
Thread 表示一个执行线程的对象 
Lock 锁原语对象（和 thread 模块中的锁一样） 
RLock 可重入锁对象，使单一线程可以（再次）获得已持有的锁（递归锁） 
Condition 条件变量对象，使得一个线程等待另一个线程满足特定的“条件”，比如改变状态或 某个数据值 
Event 条件变量的通用版本，任意数量的线程等待某个事件的发生，在该事件发生后所有 线程将被激活 
Semaphore 为线程间共享的有限资源提供了一个“计数器”，如果没有可用资源时会被阻塞 
BoundedSemaphore 与 Semaphore 相似，不过它不允许超过初始值 
Timer 与 Thread 相似，不过它要在运行前等待一段时间 
Barrier① 创建一个“障碍”，必须达到指定数量的线程后才可以继续 

#### Thread 类对象的属性和方法 

Thread 对象数据属性 name 线程名 
ident 线程的标识符 
daemon 布尔标志，表示这个线程是否是守护线程 

Thread 对象方法 
_init_(group=None, tatget=None, name=None, args=(), kwargs ={}, verbose=None, daemon=None) ③ 实例化一个线程对象，需要有一个可调用的 target，以及其参数 args 或 kwargs。还可以传递 name 或 group 参数，不过后者还未实现。此 外，verbose 标志也是可接受的。而 daemon 的值将会设定 thread.daemon 属性/标志 

In [ ]:
start() 开始执行该线程 
run() 定义线程功能的方法（通常在子类中被应用开发者重写） 
join (timeout=None) 直至启动的线程终止之前一直挂起；除非给出了 timeout（秒），否则 会一直阻塞 
getName()① 返回线程名 setName (name)① 设定线程名 
isAlivel /is_alive ()② 布尔标志，表示这个线程是否还存活 
isDaemon()③ 如果是守护线程，则返回 True；否则，返回 False 
setDaemon(daemonic)③ 把线程的守护标志设定为布尔值 daemonic（必须在线程 start()之前 调用） ① 

① 该方法已弃用，更好的方式是设置（或获取）thread.name属性，或者在实例化过程中传递该属性。 
② 驼峰式命名已经弃用，并且从 Python 2.6 版本起已经开始被取代。 
③ is/setDaemon()已经弃用，应当设置 thread.daemon 属性；从 Python 3.3 版本起，也可以通过可选的   daemon 值在实例化过 程中设定 thread.daemon 属性。 

使用 Thread 类，可以有很多方法来创建线程：
• 创建 Thread 的实例，传给它一个函数。 
• 创建 Thread 的实例，传给它一个可调用的类实例。 
• 派生 Thread 的子类，并创建子类的实例。 

你会发现你将选择第一个或第三个方案。当你需要一个更加符合面向对象的接口时， 会选择后者；否则会选择前者。老实说，你会发现第二种方案显得有些尴尬并且稍微难以 阅读

#### 创建 Thread 的实例，传给它一个函数 

In [3]:
import threading
from time import sleep, ctime

loops = [ 4, 2 ]

def loop(nloop, nsec):
    print ('start loop', nloop, 'at:', ctime())
    sleep(nsec)
    print ('loop', nloop, 'done at:', ctime())

def main():
    print ('starting at:', ctime())
    threads = []
    nloops = range(len(loops))

    for i in nloops:
        t = threading.Thread(target=loop,
	    args=(i, loops[i]))
        threads.append(t)

    for i in nloops:            # start threads
        threads[i].start()

    for i in nloops:            # wait for all
        threads[i].join()       # threads to finish

    print ('all DONE at:', ctime())

if __name__ == '__main__':
    main()


starting at: Fri Jul  7 14:51:44 2017
start loop 0 at: Fri Jul  7 14:51:44 2017
start loop 1 at: Fri Jul  7 14:51:44 2017
loop 1 done at: Fri Jul  7 14:51:46 2017
loop 0 done at: Fri Jul  7 14:51:48 2017
all DONE at: Fri Jul  7 14:51:48 2017


#### 创建 Thread 的实例，传给它一个可调用的类实例 

In [6]:
import threading
from time import sleep, ctime

loops = [ 4, 2 ]

class ThreadFunc(object):

    def __init__(self, func, args, name=''):
        self.name = name
        self.func = func
        self.args = args

    def __call__(self):
        self.func(*self.args)  # *表示实际参数不定

def loop(nloop, nsec):
    print ('start loop', nloop, 'at:', ctime())
    sleep(nsec)
    print ('loop', nloop, 'done at:', ctime())

def main():
    print ('starting at:', ctime())
    threads = []
    nloops = range(len(loops))

    for i in nloops:	# create all threads
        t = threading.Thread(
	    target=ThreadFunc(loop, (i, loops[i]),
	    loop.__name__))    #当创建新线程时，Thread 类的代码将调用 ThreadFunc 对象，此时会调用__call__()这个 特殊方法
        threads.append(t)

    for i in nloops:	# start all threads
        threads[i].start()

    for i in nloops:	# wait for completion
        threads[i].join()

    print ('all DONE at:', ctime())

if __name__ == '__main__':
    main()


starting at: Fri Jul  7 15:00:06 2017
start loop 0 at: Fri Jul  7 15:00:06 2017
start loop 1 at: Fri Jul  7 15:00:06 2017
loop 1 done at: Fri Jul  7 15:00:08 2017
loop 0 done at: Fri Jul  7 15:00:10 2017
all DONE at: Fri Jul  7 15:00:10 2017


#### 派生 Thread 的子类，并创建子类的实例 

In [7]:
import threading
from time import sleep, ctime

loops = [ 4, 2 ]

class MyThread(threading.Thread):
    def __init__(self, func, args, name=''):
        threading.Thread.__init__(self)
        self.name = name
        self.func = func
        self.args = args

    def run(self):
        self.func(*self.args)

def loop(nloop, nsec):
    print ('start loop', nloop, 'at:', ctime())
    sleep(nsec)
    print ('loop', nloop, 'done at:', ctime())

def main():
    print ('starting at:', ctime())
    threads = []
    nloops = range(len(loops))

    for i in nloops:
        t = MyThread(loop, (i, loops[i]),
	    loop.__name__)
        threads.append(t)

    for i in nloops:
        threads[i].start()

    for i in nloops:
        threads[i].join()

    print ('all DONE at:', ctime())

if __name__ == '__main__':
    main()


starting at: Fri Jul  7 15:05:05 2017
start loop 0 at: Fri Jul  7 15:05:05 2017
start loop 1 at: Fri Jul  7 15:05:05 2017
loop 1 done at: Fri Jul  7 15:05:07 2017
loop 0 done at: Fri Jul  7 15:05:09 2017
all DONE at: Fri Jul  7 15:05:09 2017


#### threading 模块的其他函数 

In [ ]:
activeCount/ active_count()① 当前活动的 Thread 对象个数 
current Thread() /current_thread① 返回当前的 Thread 对象 enumerate() 返回当前活动的 Thread 对象列表 
settrace (func) ② 为所有线程设置一个 trace 函数 
setprofile (func) ② 为所有线程设置一个 profile 函数 
stack_size (size=0) ③ 返回新创建线程的栈大小；或为后续创建的线程设定栈的大小 为 size

① 驼峰式命名已经弃用，并且从 Python 2.6 版本起已经开始被取代。 
② 自 Python 2.3 版本开始引入。 
③ thread.stack_size()的一个别名，（都是）从 Python 2.5 版本开始引入的。 

### 单线程和多线程运行比较

In [8]:
import threading
from time import time, ctime

class MyThread(threading.Thread):
    def __init__(self, func, args, name=''):
        threading.Thread.__init__(self)
        self.name = name
        self.func = func
        self.args = args

    def getResult(self):
        return self.res

    def run(self):
        print ('starting', self.name, 'at:', ctime())
        self.res = self.func(*self.args)
        print (self.name, 'finished at:', ctime())

In [11]:
from time import ctime, sleep

def fib(x):
    sleep(0.005)
    if x < 2: return 1
    return (fib(x-2) + fib(x-1))

def fac(x):
    sleep(0.1)
    if x < 2: return 1
    return (x * fac(x-1))

def sum(x):
    sleep(0.1)
    if x < 2: return 1
    return (x + sum(x-1))

funcs = (fib, fac, sum)
n = 12

def main():
    nfuncs = range(len(funcs))

    print ('*** SINGLE THREAD')
    for i in nfuncs:
        print ('starting', funcs[i].__name__, 'at:', ctime())
        print (funcs[i](n))
        print (funcs[i].__name__, 'finished at:', ctime())

    print ('\n*** MULTIPLE THREADS')
    threads = []
    for i in nfuncs:
        t = MyThread(funcs[i], (n,),
	    funcs[i].__name__)
        threads.append(t)

    for i in nfuncs:
        threads[i].start()

    for i in nfuncs:
        threads[i].join()
        print (threads[i].getResult())

    print ('all DONE')

if __name__ == '__main__':
    main()


*** SINGLE THREAD
starting fib at: Fri Jul  7 15:16:00 2017
233
fib finished at: Fri Jul  7 15:16:03 2017
starting fac at: Fri Jul  7 15:16:03 2017
479001600
fac finished at: Fri Jul  7 15:16:04 2017
starting sum at: Fri Jul  7 15:16:04 2017
78
sum finished at: Fri Jul  7 15:16:05 2017

*** MULTIPLE THREADS
starting fib at: Fri Jul  7 15:16:05 2017
starting fac at: Fri Jul  7 15:16:05 2017
starting sum at: Fri Jul  7 15:16:05 2017
fac finished at: Fri Jul  7 15:16:06 2017
sum finished at: Fri Jul  7 15:16:06 2017
fib finished at: Fri Jul  7 15:16:08 2017
233
479001600
78
all DONE


### Queue/queue 模块 

#### Queue/queue 模块常用属性 

Queue/queue 模块的类 
Queue(maxsize=0) 创建一个先入先出队列。如果给定最大值，则在队列没有空间时阻塞；否则（没 有指定最大值），为无限队列 
LifoQueue(maxsize=0) 创建一个后入先出队列。如果给定最大值，则在队列没有空间时阻塞;否则（没 有指定最大值），为无限队列  
PriorityQueue(maxsize=0) 创建一个优先级队列。如果给定最大值，则在队列没有空间时阻塞，否则（没 有指定最大值）,为无限队列 

Queue/queue 异常 
Empty  当对空队列调用 get*()方法时抛出异常 
Full  当对已满的队列调用 put*()方法时抛出异常 

Queue/queue 对象方法 
qsize () 返回队列大小（由于返回时队列大小可能被其他线程修改，所以该值为近似值） 
empty() 如果队列为空，则返回 True；否则，返回 False full() 如果队列已满，则返回 True；否则，返回 False 
put (item, block=Ture, timeout=None) 将 item 放入队列。如果 block 为 True（默认）且 timeout 为 None，则在有可用 空间之前阻塞；如果timeout为正值，则最多阻塞timeout秒；如果 block为False， 则抛出 Empty异常 
put_nowait(item) 和 put(item, False)相同 
get (block=True, timeout=None) 从队列中取得元素。如果给定了 block（非 0），则一直阻塞到有可用的元素 为止 
get_nowait() 和 get(False)相同 
task_done() 用于表示队列中的某个元素已执行完成，该方法会被下面的 join()使用 
join() 在队列中所有元素执行完毕并调用上面的 task_done()信号之前，保持阻塞 

In [14]:
from random import randint
from time import sleep
from queue import Queue
#from myThread import MyThread

def writeQ(queue):
    print ('producing object for Q...',)
    queue.put('xxx', 1)
    print ("size now", queue.qsize())

def readQ(queue):
    val = queue.get(1)
    print ('consumed object from Q... size now', queue.qsize())

def writer(queue, loops):
    for i in range(loops):
        writeQ(queue)
        sleep(randint(1, 3))

def reader(queue, loops):
    for i in range(loops):
        readQ(queue)
        sleep(randint(2, 5))

funcs = [writer, reader]
nfuncs = range(len(funcs))

def main():
    nloops = randint(2, 5)
    q = Queue(32)

    threads = []
    for i in nfuncs:
        t = MyThread(funcs[i], (q, nloops), \
            funcs[i].__name__)
        threads.append(t)

    for i in nfuncs:
        threads[i].start()

    for i in nfuncs:
        threads[i].join()

    print ('all DONE')

if __name__ == '__main__':
    main()


starting writer at: Fri Jul  7 16:37:08 2017
producing object for Q...
size now 1
starting reader at: Fri Jul  7 16:37:08 2017
consumed object from Q... size now 0
producing object for Q...
size now 1
consumed object from Q... size now 0
producing object for Q...
size now 1
consumed object from Q... size now 0
producing object for Q...
size now 1
producing object for Q...
size now 2
consumed object from Q... size now 1
writer finished at: Fri Jul  7 16:37:20 2017
consumed object from Q... size now 0
reader finished at: Fri Jul  7 16:37:26 2017
all DONE
